In [ ]:
"""
Download miniseed records from IRIS for a given selection of stations.
Miniseed data are saved in day-long chunks.

INPUT:
- CSV file containing station names, networks, and channels (format: "STA.NET.mseed,HHE,HHN,HHZ")

OUTPUT:
- Day-long miniseed files for the given period for each station.

"""

In [ ]:
import obspy
from obspy.clients.fdsn.mass_downloader import RectangularDomain, \
    Restrictions, MassDownloader
from obspy.clients.fdsn import Client
import numpy as np


In [ ]:
# Download non-restricted data from IRIS
client = Client('IRIS')
# Rectangular domain containing NW Oregon.
domain = RectangularDomain(minlatitude=44.5, maxlatitude=46.5,
                           minlongitude=-124.5, maxlongitude=-121.5)

# Read in station and network info
sdat=np.loadtxt("fname_PortlandStats.csv",skiprows=1,delimiter=',',dtype='str')

# Download all channels of data for all stations in the given time period
for i in range(len(sdat)):
    stainfo=sdat[i,0].split('.')
    chan=[*sdat[i,1]]
    restrictions = Restrictions(
        # Get data for the deployment period.
        starttime=obspy.UTCDateTime(2022, 5, 27),
        endtime=obspy.UTCDateTime(2022, 7, 3),
        # Chunk it to have one file per day.
        chunklength_in_sec=86400,
        # station info
        network=stainfo[1], station=stainfo[0], location="*", channel=chan[0]+chan[1]+"*",
        # Keep data with gaps
        reject_channels_with_gaps=False,
        # Keep data with small data lengths
        minimum_length=0.0,
        # Guard against the same station having different names.
        minimum_interstation_distance_in_m=100.0,
        # If a station does not have XML data, don't delete all its downloaded data
        sanitize=False,
    )

    mdl = MassDownloader(providers=[client])
    # save mseeds to "PNSN_data" directory, and StationXML files to "stations" directory
    mdl.download(domain, restrictions, mseed_storage="PNSN_data",
                 stationxml_storage="stations")